## Prueba de OpenScoring

### Setup

Antes hay que instalar y levantar el servicio que está implementado en Java.

```
version=2.1-SNAPSHOT
cd openscoring-server/target
java -jar openscoring-server-executable-${version}.jar --port 8181

```

La librería que usa el cliente se instala con:

```
pip install --upgrade git+https://github.com/sebastianf101/openscoring-python.git
```

In [1]:
from openscoring import Openscoring

os = Openscoring(base_url = "http://localhost:8181/openscoring")

In [2]:
os.deployFile("Iris", "openscoring/tests/resources/DecisionTreeIris.pmml")



In [3]:
arguments = {
	"Sepal.Length" : 5.1,
	"Sepal.Width" : 3.5,
	"Petal.Length" : 1.4,
	"Petal.Width" : 0.2
}

results = os.evaluate("Iris", arguments)
print(results)

{'Species': 'setosa', 'probability(setosa)': 1.0, 'probability(versicolor)': 0.0, 'probability(virginica)': 0.0}


In [4]:
from openscoring import EvaluationRequest

evaluationRequest = EvaluationRequest("record-001", arguments)

evaluationResponse = os.evaluate("Iris", evaluationRequest)
print(evaluationResponse.results)

{'Species': 'setosa', 'probability(setosa)': 1.0, 'probability(versicolor)': 0.0, 'probability(virginica)': 0.0}


In [7]:
os.evaluateCsvFile("Iris", "openscoring/tests/resources/input.csv", "openscoring/tests/resources/output.csv")

In [8]:
import pandas as pd

# Read the CSV file into a Pandas DataFrame
df = pd.read_csv("openscoring/tests/resources/output.csv")

# Print the DataFrame
print(df)


           Id     Species  probability(setosa)  probability(versicolor)  \
0  record-001      setosa                  1.0                 0.000000   
1  record-002  versicolor                  0.0                 0.907407   
2  record-003   virginica                  0.0                 0.021739   

   probability(virginica)  
0                0.000000  
1                0.092593  
2                0.978261  
